In [1]:
from pyspark.sql import SparkSession
import getpass

username = getpass.getuser()

spark = SparkSession. \
    builder. \
    config('spark.ui.port','0'). \
    config("spark.sql.warehouse.dir", f"/user/{username}/warehouse"). \
    enableHiveSupport(). \
    master('yarn'). \
    getOrCreate()

In [2]:
orders_rdd = spark.sparkContext.textFile("/user/itv019463/TrendyTechBigData/Week4/data/orders_1gb.csv")

In [3]:
customers_rdd = spark.sparkContext.textFile("/user/itv019463/TrendyTechBigData/Week4/data/part-customers")

In [4]:
customers_mapped = customers_rdd.map(lambda x: (x.split(",")[0], x.split(",")[8]))

In [5]:
customers_broadcast = spark.sparkContext.broadcast(customers_mapped.collect())

In [6]:
def get_pincode(cust_id):
    try:
        return customers_broadcast.value[cust_id]
    except:
        return "-1"

In [7]:
orders_mapped = orders_rdd.map(lambda x: (x.split(",")[2], x.split(",")[3]))

In [8]:
joined_rdd = orders_mapped.map(lambda x: (get_pincode(int(x[0])), x[1]))

In [9]:
joined_rdd.saveAsTextFile("/user/itv019463/TrendyTechBigData/Week4/data/outputBroadcast/")

In [10]:
spark.stop()